In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
%run FuzzyLogic.ipynb

{':', 'а', 'l', '"', 'й', 'ц', 'с', 'к', '-', 'j', 'ь', 'ч', 'г', '+', '_', 'q', '{', '<', '*', '%', 'ж', 'х', 'в', ')', '/', 'i', 'f', 'b', 'g', ' ', '3', 'p', 'л', 'е', ',', 'ъ', ';', '\\', 'ф', 'щ', 'c', 'a', 'k', 'ш', '8', '7', '0', 'z', 'б', 'у', 'r', 'ы', 'я', 'ю', '!', '6', 'e', '2', 'н', 'u', '=', 'м', '4', 'd', 'y', 'x', 'з', 't', 'э', 'п', '1', 'д', 'р', '.', 'ё', '~', 'и', 'm', '5', '9', 'n', 'v', '(', 'w', 's', 'h', 'о', 'o', 'т', '№'}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]


In [3]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [597]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [693]:
data = ["Ты пидор!", "Знай, Ты пидор!", "Знай, пидор!"]
data = [s.lower() for s in data];

look_back = 7
batch_size = 32
num_lstm = 5

x = []
for s in data:
    sx = []
    for c in list(s):
        vec = onehot_encoded[char_to_int[c]]
        sx.append(vec)
    x.append(sx)
    
dataframe = pandas.DataFrame.from_records(x)
dataset = dataframe.values
#dataset = dataset.astype('float32')
    
print(dataset.shape)

train_size = int(len(dataset) * 0.67)

train = dataset[:train_size];
test = dataset[train_size:]


    
    
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(train, look_back)
    
print(trainX.shape)
    
trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[2]))
trainY = numpy.reshape(trainY, (trainY.shape[0], trainY.shape[1]))

testX = numpy.reshape(testX, (testX.shape[0], look_back, testX.shape[2]))
testY = numpy.reshape(testY, (testY.shape[0], testY.shape[1]))
    

(3, 15)
(0,)


IndexError: tuple index out of range

In [647]:
# reshape into X=t and Y=t+1
look_back = 7
batch_size = 1
num_lstm = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
print(trainX.shape)
print(trainY.shape)

#trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[2]))
trainY = numpy.reshape(trainY, (trainY.shape[0], trainY.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], look_back, testX.shape[2]))
testY = numpy.reshape(testY, (testY.shape[0], testY.shape[1]))

print(trainX)
print(trainX.shape)
print(trainY.shape)

(36, 7, 90)
(36, 90)
[[[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

In [696]:
# load the dataset
data = ["Ты пидор!Знай,Ты пидор!Пидор", "Ты пидор!Знай,Ты пидор!Пидор", "Ms1 SM1MS1SM11MS MS", 
        "MS1MS1MS1MS1MS1"]
data = [s.lower() for s in data];

look_back = 7
batch_size = 1
num_lstm = 3

model = Sequential()
model.add(LSTM(90, return_sequences=True, input_shape=(look_back, 90)))
for i in range(2, num_lstm):
    model.add(LSTM(90, return_sequences=True))
model.add(LSTM(90))
model.add(Dense(90, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='rmsprop')

for s in data: 
    x = []
    for c in list(s):
        vec = onehot_encoded[char_to_int[c]]
        x.append(vec)

    dataframe = pandas.DataFrame.from_records(x)
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    
    # normalize the dataset
#    scaler = MinMaxScaler(feature_range=(0, 1))
#    dataset = scaler.fit_transform(dataset)
    
#    train_size = int(len(dataset) * 0.67)
    train = dataset
    trainX, trainY = create_dataset(train, look_back)
    
    trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[2]))
    trainY = numpy.reshape(trainY, (trainY.shape[0], trainY.shape[1]))
    
    print(trainX.shape)
    print(trainY.shape)
    
    model.fit(trainX, trainY, epochs=20, batch_size=batch_size, shuffle=False, verbose=2)
    model.reset_states()

(20, 7, 90)
(20, 90)
Epoch 1/20
 - 67s - loss: 0.0110
Epoch 2/20
 - 1s - loss: 0.0110
Epoch 3/20
 - 1s - loss: 0.0107
Epoch 4/20
 - 1s - loss: 0.0105
Epoch 5/20
 - 1s - loss: 0.0104
Epoch 6/20
 - 1s - loss: 0.0104
Epoch 7/20
 - 1s - loss: 0.0103
Epoch 8/20
 - 1s - loss: 0.0103
Epoch 9/20
 - 1s - loss: 0.0102
Epoch 10/20
 - 1s - loss: 0.0102
Epoch 11/20
 - 1s - loss: 0.0101
Epoch 12/20
 - 1s - loss: 0.0100
Epoch 13/20
 - 1s - loss: 0.0099
Epoch 14/20
 - 1s - loss: 0.0098
Epoch 15/20
 - 1s - loss: 0.0096
Epoch 16/20
 - 1s - loss: 0.0095
Epoch 17/20
 - 0s - loss: 0.0094
Epoch 18/20
 - 1s - loss: 0.0093
Epoch 19/20
 - 1s - loss: 0.0092
Epoch 20/20
 - 1s - loss: 0.0091
(20, 7, 90)
(20, 90)
Epoch 1/20
 - 1s - loss: 0.0090
Epoch 2/20
 - 1s - loss: 0.0089
Epoch 3/20
 - 1s - loss: 0.0087
Epoch 4/20
 - 1s - loss: 0.0086
Epoch 5/20
 - 1s - loss: 0.0082
Epoch 6/20
 - 1s - loss: 0.0082
Epoch 7/20
 - 0s - loss: 0.0081
Epoch 8/20
 - 0s - loss: 0.0078
Epoch 9/20
 - 0s - loss: 0.0076
Epoch 10/20
 - 1s 

In [697]:
for s in data:
    x = []
    for c in list(s):
        vec = onehot_encoded[char_to_int[c]]
        x.append(vec)

    dataframe = pandas.DataFrame.from_records(x)
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    
    # normalize the dataset
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #dataset = scaler.fit_transform(dataset)
    
    
    train = dataset
    trainX, trainY = create_dataset(train, look_back)
    
    trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[2]))
    trainY = numpy.reshape(trainY, (trainY.shape[0], trainY.shape[1]))
    
    trainPredict = model.predict(trainX, batch_size=1)
    
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform(trainY)
    
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Train Score: %.2f RMSE' % (trainScore))
    
    for p in trainPredict:
        max_i = 0
        max_v = 0
        for i, v in enumerate(p.tolist()):
            if v > max_v:
                max_i = i
                max_v = v
        print(int_to_char[max_i])

Train Score: 0.12 RMSE
р
s
1
s
s
s
,
,
д
д
п
о
о
s
р
s
1
s
s
s
Train Score: 0.12 RMSE
р
s
1
s
s
s
,
,
д
д
п
о
о
s
р
s
1
s
s
s
Train Score: 0.14 RMSE
s
1
m
1
s
s
m
s
1
m
1
Train Score: 0.00 RMSE
s
1
m
s
1
m
s


In [649]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(90, return_sequences=True, batch_input_shape=(batch_size, look_back, 90), stateful=True))
for i in range(2, num_lstm):
    model.add(LSTM(90, return_sequences=True, stateful=True))
model.add(LSTM(90))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=batch_size, shuffle=False, verbose=2)

Epoch 1/20
 - 60s - loss: 0.0108
Epoch 2/20
 - 0s - loss: 0.0102
Epoch 3/20
 - 0s - loss: 0.0100
Epoch 4/20
 - 0s - loss: 0.0099
Epoch 5/20
 - 0s - loss: 0.0098
Epoch 6/20
 - 0s - loss: 0.0097
Epoch 7/20
 - 0s - loss: 0.0097
Epoch 8/20
 - 0s - loss: 0.0098
Epoch 9/20
 - 0s - loss: 0.0098
Epoch 10/20
 - 0s - loss: 0.0100
Epoch 11/20
 - 0s - loss: 0.0106
Epoch 12/20
 - 0s - loss: 0.0097
Epoch 13/20
 - 0s - loss: 0.0095
Epoch 14/20
 - 0s - loss: 0.0097
Epoch 15/20
 - 0s - loss: 0.0100
Epoch 16/20
 - 0s - loss: 0.0099
Epoch 17/20
 - 0s - loss: 0.0096
Epoch 18/20
 - 0s - loss: 0.0097
Epoch 19/20
 - 0s - loss: 0.0099
Epoch 20/20
 - 0s - loss: 0.0096


In [650]:
# make predictions
trainPredict = model.predict(trainX, batch_size=1)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

ValueError: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 36 samples. Batch size: 32.

In [ ]:
for p in trainPredict:
    max_i = 0
    max_v = 0
    for i, v in enumerate(p.tolist()):
        if v > max_v:
            max_i = i
            max_v = v
    print(int_to_char[max_i])

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)jk 
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()